In [1]:
import os
import os.path
os.chdir('..')

In [2]:
import torch
import numpy as np
from pointnet_module import SupervisedPointNet
from datasets.shapenet_parts.shapenet_parts import ShapeNetParts
from util.visualization_utils import visualize_pointcloud
from matplotlib import cm, colors


/home/mert/venvs/ml43d/lib/python3.7/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


In [3]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    new_y = torch.eye(num_classes)[y.cpu().data.numpy(),]
    if (y.is_cuda):
        return new_y.cuda()
    return new_y

In [4]:
cwd = os.getcwd()
print(cwd)

model = SupervisedPointNet.load_from_checkpoint('supervised_best.ckpt')

/home/mert/Documents/projects/ml43d/3d-object-part-segmentation-with-simclr


In [72]:
test_dataset = ShapeNetParts('test', transforms=None)

sample, gt_seg, gt_cls = test_dataset[700]
print(sample.shape)

shape_points = sample.T

point_labels = (gt_seg - min(gt_seg)) / (max(gt_seg) - min(gt_seg))
point_colors = cm.get_cmap('hsv')(point_labels)[:, :3]
point_colors = np.sum((point_colors * 255).astype(int) * [255*255, 255, 1], axis=1)
visualize_pointcloud(shape_points.numpy(), colors=point_colors, point_size=0.025, flip_axes=True)


torch.Size([3, 2500])


/home/mert/venvs/ml43d/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [87]:
model.eval()
model.freeze()

sample, gt_seg, gt_cls = test_dataset[700]

gt_cls_tensor = torch.tensor([gt_cls])

with torch.no_grad():
    prediction = model.inference_step(sample.unsqueeze(0), gt_cls_tensor)
print(prediction.shape)

torch.Size([2500])


In [88]:
shape_points = sample.T

point_labels = (prediction - min(prediction)) / (max(prediction) - min(prediction))
point_colors = cm.get_cmap('hsv')(point_labels)[:, :3]
point_colors = np.sum((point_colors * 255).astype(int) * [255*255, 255, 1], axis=1)
visualize_pointcloud(shape_points.numpy(), colors=point_colors, point_size=0.025, flip_axes=True)

Output()